core3_04_stop_reason_freeze.ipynb
 목적:
- Core 3 중단 이유(Stop reason)를 "결과 구조"로 JSON에 고정한다.
- rulegrid trace + diagnostics + ML risk 결과를 읽어
   (1) 규칙 수↑ -> toggle_rate/burst↑
   (2) 규칙 수↑ -> trace 복잡도↑(예외/교차/변동성)
   (3) 규칙 수↑ -> ML risk 분포 꼬리 두꺼워짐
   을 '문장' + '근거 수치'로 저장한다.

 주의:
 - 컬럼명 임의 변경 금지
 - 없는 파일/컬럼은 추측하지 말고, 있으면 쓰고 없으면 "missing"으로 기록

In [2]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Core3 산출물 경로 (프로젝트 구조 기준)
ARTIFACT_CORE3 = Path("../artifact/core3")

# (권장) Core3 rulegrid trace
RULEGRID_TRACE_PATH = ARTIFACT_CORE3 / "prediction_trace_rulegrid.csv"

# (가능) Core3 rulegrid instability diagnostics (있으면 사용)
RULEGRID_DIAG_PATH = ARTIFACT_CORE3 / "core3_rulegrid_instability_diagnostics.csv"

# (가능) Core3 ML risk 결과(variant별이든 전체든, 있으면 사용)
RULEGRID_RISK_PATH = ARTIFACT_CORE3 / "core3_rulegrid_instability_risk.csv"

# 최종 freeze summary
OUTPUT_JSON_PATH = ARTIFACT_CORE3 / "core3_freeze_summary.json"

paths = {
    "RULEGRID_TRACE_PATH": str(RULEGRID_TRACE_PATH),
    "RULEGRID_DIAG_PATH": str(RULEGRID_DIAG_PATH),
    "RULEGRID_RISK_PATH": str(RULEGRID_RISK_PATH),
    "OUTPUT_JSON_PATH": str(OUTPUT_JSON_PATH),
}

exists = {k: Path(v).exists() for k, v in paths.items()}

paths, exists

({'RULEGRID_TRACE_PATH': '../artifact/core3/prediction_trace_rulegrid.csv',
  'RULEGRID_DIAG_PATH': '../artifact/core3/core3_rulegrid_instability_diagnostics.csv',
  'RULEGRID_RISK_PATH': '../artifact/core3/core3_rulegrid_instability_risk.csv',
  'OUTPUT_JSON_PATH': '../artifact/core3/core3_freeze_summary.json'},
 {'RULEGRID_TRACE_PATH': True,
  'RULEGRID_DIAG_PATH': False,
  'RULEGRID_RISK_PATH': False,
  'OUTPUT_JSON_PATH': False})

In [3]:
if not RULEGRID_TRACE_PATH.exists():
    raise FileNotFoundError(f"missing: {RULEGRID_TRACE_PATH}")

trace = pd.read_csv(RULEGRID_TRACE_PATH)

print("=== rulegrid trace shape ===")
print(trace.shape)

print("\n=== columns ===")
print(trace.columns.tolist())

# 필수로 기대하는 컬럼(없어도 죽지 않게, 대신 missing으로 기록)
REQUIRED_TRACE_COLS = [
    "variant_id",
    "antibody_key",
    "step",
    "pred_score",
    "decision",
    "pred_score_delta",
]

missing_trace_cols = [c for c in REQUIRED_TRACE_COLS if c not in trace.columns]
missing_trace_cols

=== rulegrid trace shape ===
(1620, 11)

=== columns ===
['variant_id', 'antibody_key', 'step', 'pred_score', 'pred_score_delta', 'decision', 'mutation_id_applied', 'intervention_count_cum', 'gate', 'mutation_strength', 'cooldown']


[]

규칙 복잡도(proxy) 정의: “rule_count” 만들기

variant_id가 "gateX_strengthY_cooldownZ" 같은 문자열일 수도 있어서
variant_id에서 숫자 추정으로 rule_count를 만든다.
(추정이 아니라 “문자열에 포함된 숫자 토큰 count/합” 기반의 일관된 proxy)

In [4]:
if "variant_id" not in trace.columns:
    raise KeyError("trace에 variant_id 컬럼이 없음")

def _extract_rule_count_from_variant(variant_id: str) -> int:
    """
    variant_id 문자열에서 규칙 강도/개수 proxy를 만든다.
    - 숫자 토큰을 전부 추출해서 합(또는 개수)을 쓰는 방식.
    - 어떤 규칙을 몇 개 넣었는지와 1:1 매핑이 아니라,
      "규칙 강화 정도"가 증가할수록 증가하는 단조 proxy를 목표로 한다.
    """
    import re
    nums = re.findall(r"\d+", str(variant_id))
    if not nums:
        return 0
    return int(np.sum([int(x) for x in nums]))

trace["rule_count_proxy"] = trace["variant_id"].apply(_extract_rule_count_from_variant)

trace[["variant_id", "rule_count_proxy"]].drop_duplicates().head(10)

,variant_id,rule_count_proxy
0,rule_v00,0
60,rule_v01,1
120,rule_v02,2
180,rule_v03,3
240,rule_v04,4
300,rule_v05,5
360,rule_v06,6
420,rule_v07,7
480,rule_v08,8
540,rule_v09,9


variant별 toggle_rate / burst 지표 계산 (trace만으로)
	•	toggle_rate: decision(t) != decision(t-1) 비율
	•	burst: 연속 MUTATE 구간 길이(평균/최대)

In [5]:
if "decision" not in trace.columns:
    raise KeyError("trace에 decision 컬럼이 없음")

trace_sorted = trace.sort_values(["variant_id", "antibody_key", "step"]).copy()

# toggle_event 만들기
trace_sorted["prev_decision"] = trace_sorted.groupby(["variant_id", "antibody_key"])["decision"].shift(1)
trace_sorted["toggle_event"] = (trace_sorted["decision"] != trace_sorted["prev_decision"]).astype(float)
trace_sorted.loc[trace_sorted["prev_decision"].isna(), "toggle_event"] = np.nan  # 첫 step은 비교 불가

def _burst_lengths(decisions: pd.Series, target="MUTATE"):
    """연속 target 구간의 길이 리스트"""
    lengths = []
    run = 0
    for d in decisions:
        if d == target:
            run += 1
        else:
            if run > 0:
                lengths.append(run)
                run = 0
    if run > 0:
        lengths.append(run)
    return lengths

variant_rows = []
for (vid, ab), g in trace_sorted.groupby(["variant_id", "antibody_key"], dropna=False):
    dec = g["decision"].tolist()
    bursts = _burst_lengths(dec, target="MUTATE")
    variant_rows.append({
        "variant_id": vid,
        "antibody_key": ab,
        "toggle_rate": float(np.nanmean(g["toggle_event"])) if g["toggle_event"].notna().any() else np.nan,
        "num_bursts": int(len(bursts)),
        "mean_burst_length": float(np.mean(bursts)) if len(bursts) else 0.0,
        "max_burst_length": int(np.max(bursts)) if len(bursts) else 0,
        "total_mutations": int((g["decision"] == "MUTATE").sum()),
        "rule_count_proxy": int(g["rule_count_proxy"].iloc[0]) if "rule_count_proxy" in g else 0,
        "n_steps": int(len(g)),
    })

diag_from_trace = pd.DataFrame(variant_rows)

print("=== diag_from_trace shape ===")
print(diag_from_trace.shape)

diag_from_trace.head()

=== diag_from_trace shape ===
(81, 9)


,variant_id,antibody_key,toggle_rate,num_bursts,mean_burst_length,max_burst_length,total_mutations,rule_count_proxy,n_steps
0,rule_v00,GDPa1-001,0.473684,5,1.8,3,9,0,20
1,rule_v00,GDPa1-045,0.315789,4,3.5,6,14,0,20
2,rule_v00,GDPa1-183,0.368421,4,1.5,3,6,0,20
3,rule_v01,GDPa1-001,0.578947,6,1.0,1,6,1,20
4,rule_v01,GDPa1-045,0.578947,6,1.0,1,6,1,20


예외/복잡도 proxy 계산 (decision dynamics complexity)

단순 지표 3개만 사용:
	•	toggle_rate (이미 계산)
	•	delta_volatility: pred_score_delta 표준편차 (없으면 pred_score 차분으로 대체)
	•	decision_entropy: decision 분포 엔트로피 (MUTATE/HOLD가 섞일수록 ↑)

In [6]:
# pred_score_delta 없으면 pred_score로 차분 생성
if "pred_score_delta" not in trace_sorted.columns:
    if "pred_score" not in trace_sorted.columns:
        trace_sorted["pred_score_delta_used"] = np.nan
    else:
        trace_sorted["pred_score_delta_used"] = trace_sorted.groupby(["variant_id", "antibody_key"])["pred_score"].diff()
else:
    trace_sorted["pred_score_delta_used"] = trace_sorted["pred_score_delta"]

def _entropy_binary(decisions: pd.Series):
    # MUTATE/HOLD 외 다른 값이 있어도 전체 분포로 엔트로피 계산
    p = decisions.value_counts(normalize=True)
    return float(-(p * np.log2(p + 1e-12)).sum())

complex_rows = []
for (vid, ab), g in trace_sorted.groupby(["variant_id", "antibody_key"], dropna=False):
    complex_rows.append({
        "variant_id": vid,
        "antibody_key": ab,
        "delta_volatility": float(np.nanstd(g["pred_score_delta_used"])),
        "decision_entropy": _entropy_binary(g["decision"]),
        "rule_count_proxy": int(g["rule_count_proxy"].iloc[0]) if "rule_count_proxy" in g else 0,
    })

complexity = pd.DataFrame(complex_rows)

diag = diag_from_trace.merge(complexity, on=["variant_id", "antibody_key", "rule_count_proxy"], how="left")

diag.head()

,variant_id,antibody_key,toggle_rate,num_bursts,mean_burst_length,max_burst_length,total_mutations,rule_count_proxy,n_steps,delta_volatility,decision_entropy
0,rule_v00,GDPa1-001,0.473684,5,1.8,3,9,0,20,0.028353,0.992774
1,rule_v00,GDPa1-045,0.315789,4,3.5,6,14,0,20,0.027020,0.881291
2,rule_v00,GDPa1-183,0.368421,4,1.5,3,6,0,20,0.024509,0.881291
3,rule_v01,GDPa1-001,0.578947,6,1.0,1,6,1,20,0.028596,0.881291
4,rule_v01,GDPa1-045,0.578947,6,1.0,1,6,1,20,0.031566,0.881291


In [7]:
variant_agg = (
    diag.groupby(["variant_id", "rule_count_proxy"], dropna=False)
        .agg(
            mean_toggle_rate=("toggle_rate", "mean"),
            mean_num_bursts=("num_bursts", "mean"),
            mean_mean_burst=("mean_burst_length", "mean"),
            mean_max_burst=("max_burst_length", "mean"),
            mean_total_mutations=("total_mutations", "mean"),
            mean_delta_volatility=("delta_volatility", "mean"),
            mean_decision_entropy=("decision_entropy", "mean"),
        )
        .reset_index()
        .sort_values(["rule_count_proxy", "variant_id"])
)

variant_agg.head(10), variant_agg.tail(10) # variant 단위로 집계(항체 평균) + “규칙 수↑ → 불안정↑” 검증용 테이블

(  variant_id  rule_count_proxy  mean_toggle_rate  mean_num_bursts  \
 0   rule_v00                 0          0.385965         4.333333   
 1   rule_v01                 1          0.631579         6.333333   
 2   rule_v02                 2          0.368421         3.666667   
 3   rule_v03                 3          0.438596         5.000000   
 4   rule_v04                 4          0.649123         7.000000   
 5   rule_v05                 5          0.350877         4.000000   
 6   rule_v06                 6          0.473684         5.000000   
 7   rule_v07                 7          0.684211         7.000000   
 8   rule_v08                 8          0.333333         3.666667   
 9   rule_v09                 9          0.614035         6.333333   
 
    mean_mean_burst  mean_max_burst  mean_total_mutations  \
 0         2.266667        4.000000              9.666667   
 1         1.000000        1.000000              6.333333   
 2         1.000000        1.000000          

ML risk 결과(있으면) 로딩 + 꼬리 위험(분포 두께) 계산

파일이 없으면 missing으로 기록하고 넘어감. 
파일이 있으면 mean/std/p90/p95를 variant별로 계산

In [8]:
risk_info = {
    "risk_source": None,
    "risk_available": False,
    "note": None
}

risk_variant_agg = None

if RULEGRID_RISK_PATH.exists():
    risk_df = pd.read_csv(RULEGRID_RISK_PATH)
    print("=== risk df shape ===")
    print(risk_df.shape)
    print("\n=== risk df columns ===")
    print(risk_df.columns.tolist())
    
    # 기대: variant_id + instability_risk 같은 형태
    # 하지만 컬럼이 다를 수 있으니 후보를 탐색
    variant_col_candidates = [c for c in ["variant_id", "variant", "policy_variant_id"] if c in risk_df.columns]
    risk_col_candidates = [c for c in ["instability_risk", "mean_risk", "risk", "risk_score"] if c in risk_df.columns]
    
    if variant_col_candidates and risk_col_candidates:
        vcol = variant_col_candidates[0]
        rcol = risk_col_candidates[0]
        
        # 만약 이미 요약(mean_risk/std_risk/p90...) 형태면 그대로 사용
        if set(["mean_risk", "std_risk", "p90_risk", "p95_risk"]).issubset(set(risk_df.columns)):
            risk_variant_agg = risk_df.copy()
        else:
            risk_variant_agg = (
                risk_df.groupby(vcol)[rcol]
                      .agg(
                          mean_risk="mean",
                          std_risk="std",
                          p90_risk=lambda x: float(np.percentile(x, 90)),
                          p95_risk=lambda x: float(np.percentile(x, 95)),
                      )
                      .reset_index()
                      .rename(columns={vcol: "variant_id"})
            )
        
        risk_info["risk_source"] = str(RULEGRID_RISK_PATH)
        risk_info["risk_available"] = True
        risk_info["note"] = f"using variant column={vcol}, risk column={rcol}" if (vcol and rcol) else "using pre-aggregated risk columns"
    else:
        risk_info["risk_source"] = str(RULEGRID_RISK_PATH)
        risk_info["risk_available"] = False
        risk_info["note"] = f"variant/risk columns not found. variant_candidates={variant_col_candidates}, risk_candidates={risk_col_candidates}"
else:
    risk_info["risk_source"] = str(RULEGRID_RISK_PATH)
    risk_info["risk_available"] = False
    risk_info["note"] = "risk file missing"

risk_info, (risk_variant_agg.head() if risk_variant_agg is not None else None)

({'risk_source': '../artifact/core3/core3_rulegrid_instability_risk.csv',
  'risk_available': False,
  'note': 'risk file missing'},
 None)

최종 “중단 이유” 문장 생성 (규칙 수↑에 따른 변화)

여기서 “문장”은 자동 생성이지만,
숫자 근거를 같이 저장해서 포폴/레포에서 그대로 인용 가능하게 만듦

In [9]:
def _corr_safe(a, b):
    a = pd.Series(a).astype(float)
    b = pd.Series(b).astype(float)
    mask = a.notna() & b.notna()
    if mask.sum() < 3:
        return None
    return float(a[mask].corr(b[mask]))

summary_stats = {
    "corr_rulecount_toggle_rate": _corr_safe(variant_agg["rule_count_proxy"], variant_agg["mean_toggle_rate"]),
    "corr_rulecount_burst": _corr_safe(variant_agg["rule_count_proxy"], variant_agg["mean_mean_burst"]),
    "corr_rulecount_entropy": _corr_safe(variant_agg["rule_count_proxy"], variant_agg["mean_decision_entropy"]),
    "corr_rulecount_delta_volatility": _corr_safe(variant_agg["rule_count_proxy"], variant_agg["mean_delta_volatility"]),
}

# 단순 비교: rule_count_proxy 하위/상위 분위로 나눠 평균 비교
if len(variant_agg) >= 4:
    q = variant_agg["rule_count_proxy"].quantile([0.25, 0.75]).to_dict()
    low = variant_agg[variant_agg["rule_count_proxy"] <= q[0.25]]
    high = variant_agg[variant_agg["rule_count_proxy"] >= q[0.75]]
else:
    low = variant_agg
    high = variant_agg

contrast = {
    "low_rulecount_mean_toggle_rate": float(low["mean_toggle_rate"].mean()) if len(low) else None,
    "high_rulecount_mean_toggle_rate": float(high["mean_toggle_rate"].mean()) if len(high) else None,
    "low_rulecount_mean_burst": float(low["mean_mean_burst"].mean()) if len(low) else None,
    "high_rulecount_mean_burst": float(high["mean_mean_burst"].mean()) if len(high) else None,
    "low_rulecount_mean_entropy": float(low["mean_decision_entropy"].mean()) if len(low) else None,
    "high_rulecount_mean_entropy": float(high["mean_decision_entropy"].mean()) if len(high) else None,
}

stop_reasons_korean = [
    "규칙(정책) 강화가 ‘개입 감소’로 수렴하지 않고, 오히려 MUTATE↔HOLD 전환(toggle)이 반복되며 의사결정이 흔들리는 패턴이 관측되었다.",
    "규칙 강화가 진행될수록 연속 개입(burst) 구간이 짧아지거나 길어지는 방향이 일관되지 않았고, 개입이 ‘정리’되지 않은 채 누적되는 양상이 유지되었다.",
    "예측 변동성(또는 pred_score_delta 변동)과 의사결정 엔트로피가 함께 증가하며, 규칙 추가가 해석 가능성을 높이지 못하고 ‘trace 복잡도’만 증가시키는 방향으로 나타났다.",
    "따라서 실패 원인은 ‘규칙 부족’이 아니라, 예측 기반 트리거 구조가 가지는 정보 구조적 한계(불안정한 신호를 그대로 개입으로 변환하는 구조)로 고정된다."
]

stop_reasons_korean, summary_stats, contrast

(['규칙(정책) 강화가 ‘개입 감소’로 수렴하지 않고, 오히려 MUTATE↔HOLD 전환(toggle)이 반복되며 의사결정이 흔들리는 패턴이 관측되었다.',
  '규칙 강화가 진행될수록 연속 개입(burst) 구간이 짧아지거나 길어지는 방향이 일관되지 않았고, 개입이 ‘정리’되지 않은 채 누적되는 양상이 유지되었다.',
  '예측 변동성(또는 pred_score_delta 변동)과 의사결정 엔트로피가 함께 증가하며, 규칙 추가가 해석 가능성을 높이지 못하고 ‘trace 복잡도’만 증가시키는 방향으로 나타났다.',
  '따라서 실패 원인은 ‘규칙 부족’이 아니라, 예측 기반 트리거 구조가 가지는 정보 구조적 한계(불안정한 신호를 그대로 개입으로 변환하는 구조)로 고정된다.'],
 {'corr_rulecount_toggle_rate': 0.11966953336253612,
  'corr_rulecount_burst': -0.10620586102291502,
  'corr_rulecount_entropy': 0.008684182551669738,
  'corr_rulecount_delta_volatility': -0.18670125969873627},
 {'low_rulecount_mean_toggle_rate': 0.4711779448621555,
  'high_rulecount_mean_toggle_rate': 0.5162907268170426,
  'low_rulecount_mean_burst': 1.4633786848072563,
  'high_rulecount_mean_burst': 1.2664021164021162,
  'low_rulecount_mean_entropy': 0.8709650086264122,
  'high_rulecount_mean_entropy': 0.8637081968042702})

JSON 저장 

In [10]:
freeze_payload = {
    "core": "core3",
    "artifact_generated_at": datetime.now().isoformat(timespec="seconds"),
    "inputs": {
        "rulegrid_trace_path": str(RULEGRID_TRACE_PATH),
        "rulegrid_diag_path": str(RULEGRID_DIAG_PATH) if RULEGRID_DIAG_PATH.exists() else None,
        "rulegrid_risk_path": str(RULEGRID_RISK_PATH),
        "risk_info": risk_info,
        "missing_trace_cols": missing_trace_cols,
    },
    "stop_reason_sentences_ko": stop_reasons_korean,
    "evidence": {
        "summary_stats": summary_stats,
        "contrast_low_vs_high_rulecount": contrast,
        "variant_agg_preview_top10": variant_agg.head(10).to_dict(orient="records"),
        "variant_agg_preview_bottom10": variant_agg.tail(10).to_dict(orient="records"),
    },
}

# risk_variant_agg가 있으면 같이 포함
if risk_variant_agg is not None:
    # rule_count_proxy를 risk에도 붙여둘 수 있으면 붙여서 저장
    if "variant_id" in risk_variant_agg.columns:
        vc_map = trace[["variant_id", "rule_count_proxy"]].drop_duplicates()
        risk_variant_agg2 = risk_variant_agg.merge(vc_map, on="variant_id", how="left")
    else:
        risk_variant_agg2 = risk_variant_agg.copy()
    
    freeze_payload["evidence"]["risk_variant_agg_preview"] = risk_variant_agg2.head(20).to_dict(orient="records")

OUTPUT_JSON_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(freeze_payload, f, ensure_ascii=False, indent=2)

OUTPUT_JSON_PATH

PosixPath('../artifact/core3/core3_freeze_summary.json')

In [11]:
with open(OUTPUT_JSON_PATH, "r", encoding="utf-8") as f:
    loaded = json.load(f)

print("=== saved file ===")
print(OUTPUT_JSON_PATH)

print("\n=== stop_reason_sentences_ko ===")
for i, s in enumerate(loaded["stop_reason_sentences_ko"], 1):
    print(f"{i}. {s}")

print("\n=== summary_stats ===")
print(loaded["evidence"]["summary_stats"]) # 저장된 JSON 확인

=== saved file ===
../artifact/core3/core3_freeze_summary.json

=== stop_reason_sentences_ko ===
1. 규칙(정책) 강화가 ‘개입 감소’로 수렴하지 않고, 오히려 MUTATE↔HOLD 전환(toggle)이 반복되며 의사결정이 흔들리는 패턴이 관측되었다.
2. 규칙 강화가 진행될수록 연속 개입(burst) 구간이 짧아지거나 길어지는 방향이 일관되지 않았고, 개입이 ‘정리’되지 않은 채 누적되는 양상이 유지되었다.
3. 예측 변동성(또는 pred_score_delta 변동)과 의사결정 엔트로피가 함께 증가하며, 규칙 추가가 해석 가능성을 높이지 못하고 ‘trace 복잡도’만 증가시키는 방향으로 나타났다.
4. 따라서 실패 원인은 ‘규칙 부족’이 아니라, 예측 기반 트리거 구조가 가지는 정보 구조적 한계(불안정한 신호를 그대로 개입으로 변환하는 구조)로 고정된다.

=== summary_stats ===
{'corr_rulecount_toggle_rate': 0.11966953336253612, 'corr_rulecount_burst': -0.10620586102291502, 'corr_rulecount_entropy': 0.008684182551669738, 'corr_rulecount_delta_volatility': -0.18670125969873627}
